In [2]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [4]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

from collections import Counter
label_count_dict = dict(Counter(train_y_list).most_common())
print label_count_dict

{0: 881, 1: 2093, 2: 5590, 3: 674, 4: 721}


In [5]:
from sklearn.metrics import confusion_matrix
import itertools
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')
    print confusion_matrix(test_y_list, predict_y_list)

In [6]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [7]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [8]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [9]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [10]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [11]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[  5.66733100e-03   3.11773100e-04   5.35555800e-03   5.30000000e+01
   9.60000000e+01   1.30774500e-03   4.18643100e-06   1.09633100e-03
   1.80239600e-06   1.34818000e-03]
[  1.23016500e-02   9.11361400e-05   1.22105100e-02   5.20000000e+01
   2.40000000e+01   1.31725500e-03   3.23349300e-07   1.29866300e-03
   6.89166500e-06   2.62522000e-03]


In [12]:
X = train_x_list
y = train_y_list

In [13]:
# record all predict_y
full_predict_y_lists = []

# No Preprocessing

In [14]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [15]:
from hmmlearn.hmm import GMMHMM
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

# train models and get scores
models = []
scores = []
for i in range(5):
    X = np.array(train_x_lists[i])
    model = GMMHMM(n_mix=6, n_components=1).fit(X)
    models.append(model)

    score = []
    for XX in test_x_list:
        score.append(model.score(np.array(XX)))
    scores.append(score)

predict_y_list = np.argmax(np.array(scores).T, axis = 1)
calc_score(test_y_list, predict_y_list.tolist())

number of class 0 : 881
number of class 1 : 2093
number of class 2 : 5590
number of class 3 : 674
number of class 4 : 721


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/hmmlearn/hmm.py:593: RuntimeWarning: divide by zero encountered in log
  lgmm_posteriors = (np.log(g.predict_proba(X))


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.31335366716749152, 0.31600947437852722, 0.31294853062222278, None)
weighted:  (0.55377402126434072, 0.55625529853457667, 0.5542911649880925, None)
[[ 198  147  238    7   21]
 [ 117  537  798   14   42]
 [ 325  928 3790  120  214]
 [  15   20  139   25   16]
 [  42   69  362   30   43]]


In [16]:
full_predict_y_lists.append(predict_y_list)

In [17]:
# Linear Discriminant Analysis
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
predict_y_list = lda.fit(train_x_list, train_y_list).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2017-01-03 18:59:13
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35936656041590076, 0.3375721378731863, 0.34056468342636742, None)
weighted:  (0.58814106817086764, 0.63558193048322631, 0.60588664292089445, None)
[[ 198  129  256    9   19]
 [  69  487  906   18   28]
 [ 167  500 4513  145   52]
 [  11    2  151   39   12]
 [  42   27  421   45   11]]
end: 2017-01-03 18:59:14


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [18]:
full_predict_y_lists.append(predict_y_list)

# SMOTE

In [19]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

No handlers could be found for logger "imblearn.base"


number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [20]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
clf = AdaBoostClassifier()
predict_y_list = clf.fit(train_x_list_smote, train_y_list_smote).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2017-01-03 18:59:16
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.29531871782735519, 0.36913699769264224, 0.28409196919298363, None)
weighted:  (0.57493230756459557, 0.37628678696863266, 0.42608840417588595, None)
[[ 271  128  102   27   83]
 [ 269  619  325   93  202]
 [ 751  900 2002  648 1076]
 [  19    9   73   80   34]
 [  70   59  178  104  135]]
end: 2017-01-03 19:00:17


In [21]:
full_predict_y_lists.append(predict_y_list)

# Z-SCORE

In [22]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

In [23]:
# 2-Layer MLP
batch_size = 200
train_iter = mx.io.NDArrayIter(train_x_list, train_y_list, batch_size=batch_size)
test_iter = mx.io.NDArrayIter(test_x_list, test_y_list, batch_size=batch_size)

data = mx.symbol.Variable('data')
# data = mx.symbol.Dropout(data=data, p=0.2)
# fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
# act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=20)
mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')

# mx.viz.plot_network(mlp)

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

model = mx.model.FeedForward(
    ctx = mx.cpu(),       # Run on CPU
    symbol = mlp,         # Use the network we just defined
    num_epoch = 200,       # Train for 10 epochs
    learning_rate = 0.001,  # Learning rate
    optimizer = 'adam')
model.fit(
    X=train_iter,  # Training data set
    eval_data=test_iter)#,  # Testing data set. MXNet computes scores on test set every epoch
    #batch_end_callback = mx.callback.Speedometer(batch_size, 2000))  # Logging module to print out progress

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

predict_y_score_list = model.predict(test_x_list)
predict_y_list = np.argmax(predict_y_score_list, axis=1)

calc_score(test_y_list, predict_y_list)

INFO:root:Start training with [cpu(0)]


start: 2017-01-03 19:00:18


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.241
INFO:root:Epoch[0] Validation-accuracy=0.623333
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.240
INFO:root:Epoch[1] Validation-accuracy=0.632024
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.240
INFO:root:Epoch[2] Validation-accuracy=0.635595
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.243
INFO:root:Epoch[3] Validation-accuracy=0.627619
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.241
INFO:root:Epoch[4] Validation-accuracy=0.623690
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.241
INFO:root:Epoch[5] Validation-accuracy=0.619405
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.273
INFO:root:Epoch[6] Validation-accuracy=0.615952
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.226
INFO:root:Epoch[7] Validation-accuracy=0

start: 2017-01-03 19:01:30
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.34864629762724486, 0.32599156147255798, 0.32993799606905272, None)
weighted:  (0.5677804401133969, 0.60373016834201287, 0.58067406296731106, None)
[[ 132  133  302   14   30]
 [  43  424  980   11   50]
 [ 172  521 4332  174  178]
 [   6   11  130   53   15]
 [  35   35  391   41   44]]


In [24]:
full_predict_y_lists.append(predict_y_list)

In [25]:
# SVM with class_weight='balanced'
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

clf = svm.SVC(class_weight='balanced').fit(train_x_list, train_y_list)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2017-01-03 19:01:30
SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33471047772992807, 0.41088171030458698, 0.35022002324791102, None)
weighted:  (0.60335579154366337, 0.5066004602155747, 0.5389480771387325, None)
[[ 283  158  100    8   62]
 [ 192  706  484   22  104]
 [ 673  868 3013  335  488]
 [  16   13   76   82   28]
 [  81   51  233   82   99]]
end: 2017-01-03 19:02:55


In [26]:
full_predict_y_lists.append(predict_y_list)

In [27]:
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

weight = {
    0: 6.35 ** 0.5,
    1: 2.67 ** 0.5,
    2: 1,
    3: 8.3 ** 0.5,
    4: 7.75 ** 0.5
}

clf = svm.SVC(class_weight=weight, decision_function_shape = 'ovr').fit(train_x_list, train_y_list)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2017-01-03 19:02:55
SVC(C=1.0, cache_size=200,
  class_weight={0: 2.5199206336708304, 1: 1.6340134638368191, 2: 1, 3: 2.8809720581775866, 4: 2.7838821814150108},
  coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.4042626943866594, 0.3584048191916126, 0.36085679345920862, None)
weighted:  (0.60054714898129358, 0.6315853215453554, 0.60741039606006819, None)
[[ 204  146  240    6   15]
 [  72  529  895    4    8]
 [ 239  571 4403  132   32]
 [  13    5  141   51    5]
 [  52   36  385   45   28]]
end: 2017-01-03 19:04:08


In [28]:
full_predict_y_lists.append(predict_y_list)

# Z-SCORE + SMOTE

In [29]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 1: 2093, 0: 881, 4: 721, 3: 674})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4916 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 1: 2093, 0: 881, 4: 721})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4869 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 4: 5590, 1: 2093, 0: 881})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4709 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({0: 5590,

number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [30]:
# 2-Layer MLP with sigmoid&adam
batch_size = 200
train_iter = mx.io.NDArrayIter(train_x_list_smote, train_y_list_smote, batch_size=batch_size)
test_iter = mx.io.NDArrayIter(test_x_list, test_y_list, batch_size=batch_size)

data = mx.symbol.Variable('data')
# data = mx.symbol.Dropout(data=data, p=0.2)
# fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
# act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="sigmoid")
fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=5)
mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')

# mx.viz.plot_network(mlp)

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

model = mx.model.FeedForward(
    ctx = mx.cpu(),       # Run on CPU
    symbol = mlp,         # Use the network we just defined
    num_epoch = 200,       # Train for 10 epochs
    learning_rate = 0.001,  # Learning rate
    optimizer = 'adam')
model.fit(
    X=train_iter,  # Training data set
    eval_data=test_iter)#,  # Testing data set. MXNet computes scores on test set every epoch
    #batch_end_callback = mx.callback.Speedometer(batch_size, 2000))  # Logging module to print out progress

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

predict_y_score_list = model.predict(test_x_list)
predict_y_list = np.argmax(predict_y_score_list, axis=1)

calc_score(test_y_list, predict_y_list)

INFO:root:Start training with [cpu(0)]


start: 2017-01-03 19:04:18


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=1.052
INFO:root:Epoch[0] Validation-accuracy=0.184881
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=1.053
INFO:root:Epoch[1] Validation-accuracy=0.199048
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=1.053
INFO:root:Epoch[2] Validation-accuracy=0.201310
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=1.053
INFO:root:Epoch[3] Validation-accuracy=0.202500
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=1.055
INFO:root:Epoch[4] Validation-accuracy=0.198929
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=1.056
INFO:root:Epoch[5] Validation-accuracy=0.197143
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=1.055
INFO:root:Epoch[6] Validation-accuracy=0.197381
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=1.056
INFO:root:Epoch[7] Validation-accuracy=0

start: 2017-01-03 19:08:07
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.32420695235977692, 0.36037655808081326, 0.33046326132662535, None)
weighted:  (0.57187359152895878, 0.53300230107787328, 0.54969553201226429, None)
[[ 169  152  208   26   56]
 [  80  531  747   35  115]
 [ 286  693 3540  377  481]
 [  12    8   96   78   21]
 [  47   56  285   75   83]]


In [31]:
full_predict_y_lists.append(predict_y_list)

In [32]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
clf = AdaBoostClassifier()
predict_y_list = clf.fit(train_x_list_smote, train_y_list_smote).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2017-01-03 19:08:08
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.30003992955158304, 0.37901346181962825, 0.26837634614581041, None)
weighted:  (0.58961032910336686, 0.32529974567034031, 0.37646690561181945, None)
[[ 235  141   53   38  144]
 [ 212  600  267  129  300]
 [ 664  843 1564  860 1446]
 [  20    5   37  105   48]
 [  63   47  115  139  182]]
end: 2017-01-03 19:09:10


In [33]:
full_predict_y_lists.append(predict_y_list)

In [34]:
print len(full_predict_y_lists)

8


# Result

In [57]:
predict_y_lists = []
# predict_y_lists.append(full_predict_y_lists[0])
predict_y_lists.append(full_predict_y_lists[1])
# predict_y_lists.append(full_predict_y_lists[2])
predict_y_lists.append(full_predict_y_lists[3])
predict_y_lists.append(full_predict_y_lists[4])
predict_y_lists.append(full_predict_y_lists[5])
# predict_y_lists.append(full_predict_y_lists[6])
# predict_y_lists.append(full_predict_y_lists[7])

In [58]:
# FUSION
from collections import Counter

fusion_predict_y_list = []
for i in xrange(len(predict_y_list)):
    c = Counter([predict_y_lists[j][i] for j in xrange(len(predict_y_lists))])
    most_count = c.most_common(1)[0][1]
    most_label = [tup[0] for tup in c.most_common() if tup[1] == most_count]
    most_label_dict = {key: label_count_dict[key] for key in most_label}
    
    # better accuracy
#     fusion_predict_y_list.append(max(most_label_dict, key=most_label_dict.get))
    # better recall
    fusion_predict_y_list.append(min(most_label_dict, key=most_label_dict.get))

calc_score(test_y_list, fusion_predict_y_list)

Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.37776362216112513, 0.38868895853814595, 0.3752757085768969, None)
weighted:  (0.60148748202099334, 0.61232893302652291, 0.60473004539373432, None)
[[ 226  148  204    7   26]
 [  92  605  766   11   34]
 [ 295  660 4110  201  111]
 [  14    7  114   70   10]
 [  55   42  348   56   45]]


In [59]:
for i in range(len(predict_y_lists)):
    print predict_y_lists[i][:30].astype(int)
print '--------------------'
print test_y_list[:30]
print '--------------------'
print np.array(fusion_predict_y_list[:30]).astype(int)

[2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2]
[2 2 2 2 2 4 2 2 2 2 3 2 2 2 2 1 2 4 2 2 2 2 4 1 2 4 4 2 4 1]
[2 2 2 4 2 0 4 2 1 0 3 2 4 2 1 1 1 4 3 2 2 2 0 1 2 4 4 2 4 4]
[2 2 2 2 2 0 2 2 2 2 3 2 2 2 2 1 1 2 2 2 2 2 0 1 2 2 2 2 4 2]
--------------------
[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2]
--------------------
[2 2 2 2 2 0 2 2 2 2 3 2 2 2 2 1 1 4 2 2 2 2 0 1 2 4 4 2 4 2]


In [60]:
# save_result_path = 'results/v1/fusion_predict_y_list.pickle'
# pickle.dump(fusion_predict_y_list, open(save_result_path, 'w'))